In [1]:
setwd(paste0(Sys.getenv('R_SOURCES'), '/house_prices'))
getwd()

[1] "/mnt/hdd/develop/kaggle/R_sources/house_prices"

In [3]:
source('main.R')
house_prices$helpers$import_libs()
combined_dataset <- house_prices$helpers$load_data()
combined_dataset %>% dim

[1] 2919   82

In [4]:
combined_dataset_fixed <-
    combined_dataset %>%
    (house_prices$missing$categ$replace_with_most_common) %>%
    (house_prices$missing$categ$fix_valid) %>%
    (house_prices$missing$numeric$replace_with_zero)

test_that("should be no NA values except SalePrice column", {
    expect_equal(sum(is.na(combined_dataset_fixed %>% select(-SalePrice))), 0)
})